In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ls #Ver a localização desse script nas pastas

In [0]:
if (True):
  import os
  #os.chdir("drive/My Drive/")  #MAPEAR PARA O LOCAL NO DRIVE ONDE SE ENCONTRA ESSE ARQUIVO
  os.chdir("drive/My Drive/Aula3_DL/Exercícios")

In [0]:
!ls #Ver a localização desse script nas pastas

# Parte 1 - Carregar e tratar os dados

In [0]:
# Instalar pacotes necessários para a prática
#!pip install theano
#!pip install tensorflow
#!pip install keras

In [0]:
# Importando as bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
# Importando a base de dados
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values #três primeiras colunas não interessam
y = dataset.iloc[:, 13].values #útima coluna é op target

In [0]:
pd.DataFrame(X)

In [0]:
pd.DataFrame(y)

In [0]:
# Codificando atributos categóricos
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#atributo geography: três categorias: France, Spain, Germany
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

#atributo Gender: duas categorias: Male, Female
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [0]:
pd.DataFrame(X)

In [0]:
# OneHot: atributos binários para Geography: dummy coding 
# => resulta em três colunas, mas uma é retirada 
# (evita multicolinearidade: quando uma das colunas é totalmente explicada pelas outras)
# Portanto, no final, a base resultante tem somente uma coluna a mais.
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [0]:
pd.DataFrame(X)

In [0]:
# Dividir a base em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [0]:
# 8000 registros
X_train.shape

In [0]:
#2 000 registros
X_test.shape

In [0]:
# Normalização
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
X_train


# Parte 2 - Vamos criar a rede neural!

In [0]:
# Importando as bibliotecas do Keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [0]:
# Iniciando a Rede Neural
classifier = Sequential()

# Adicionando a camada de input e a primeira camada escondida
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dropout(rate = 0.1))

# Adicionando a segunda camada escondida
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

# Adicionando a camada de saída
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compilando a rede neural
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Visualização das camadas da rede
classifier.summary()

In [0]:
# Treinando a rede neural
trained_model = classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, validation_data = (X_test,y_test))

In [0]:
print(trained_model.history.keys())

In [0]:
plt.plot(trained_model.history['acc'])
plt.plot(trained_model.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
plt.plot(trained_model.history['loss'])
plt.plot(trained_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Parte 3 - Fazer previsões e avaliar a rede treinada

In [0]:
# Previsão da base de teste
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [0]:
# Matriz de confusão
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Prevendo uma observação

In [0]:
""" Prever se um cliente com as seguintes características vai sair do banco:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 1000"""
att_vector = np.array([[0.0, 0, 600, 1, 40, 3, 60, 2, 1, 1, 1000]])
att_vector_normalized = sc.transform(att_vector)
new_prediction = classifier.predict(att_vector_normalized)
new_prediction = (new_prediction > 0.5)

In [0]:
print(new_prediction)